### Challenge 24: Create the MT19937 stream cipher and break it

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

You can create a trivial stream cipher out of any PRNG; use it to generate a sequence of 8 bit outputs and call those outputs a keystream. XOR each byte of plaintext with each successive byte of keystream.

Write the function that does this for MT19937 using a 16-bit seed. Verify that you can encrypt and decrypt properly. This code should look similar to your CTR code.

Use your function to encrypt a known plaintext (say, 14 consecutive 'A' characters) prefixed by a random number of random characters.

From the ciphertext, recover the "key" (the 16 bit seed).

Use the same idea to generate a random "password reset token" using MT19937 seeded from the current time.

Write a function to check if any given password token is actually the product of an MT19937 PRNG seeded with the current time.

</div>

In [27]:
import mt19937 as mt
from random import randint
import math
import cryptopals as cp

In [33]:
def mt19937_encrypt(plaintext):

    seed = randint(0, 2**16 - 1)
    myRNG = mt.mt19937(seed)

    if isinstance(plaintext, str):
        pt_bytes = plaintext.encode()
    elif isinstance(plaintext, int):
        pt_bytes = plaintext.to_bytes()
    else:
        raise ValueError('Not sure what to do with this plaintext data type')

    pt_len = len(pt_bytes)

    n_words = math.ceil(pt_len/4)

    key_stream = []
    for __ in range(n_words):
        key_stream += myRNG.extract_number().to_bytes(4, 'little')

    ciphertext = cp.bitwise_xor(pt_bytes, key_stream[0:pt_len])
    return(ciphertext)

In [49]:
def crack_mt19937_encrypt(ciphertext):

    ct_len = len(ciphertext)
    n_words = math.ceil(ct_len/4)
    seed_scores = [0] * (2**16)
    best_seed = -1
    best_score = -1
    for seed in range(2**16 - 1):
        if seed % 2**12 == 0:
            print('.')
        key_stream = []
        myRNG = mt.mt19937(seed)
        for __ in range(n_words):
            key_stream += myRNG.extract_number().to_bytes(4, 'little')
        decrypted_data = cp.bitwise_xor(ciphertext, key_stream[0:ct_len])
        seed_scores[seed] = cp.score_english(decrypted_data)
        if seed_scores[seed] > best_score:
            best_seed = seed
            plaintext = decrypted_data
            best_score = seed_scores[seed]
            print(f"New Best:  {seed}, {best_score}")

    return(plaintext, best_seed)

In [50]:
seed = randint(0, 2**16 - 1)
myRNG = mt.mt19937(seed)

# %% Part 1
# Just brute force crack it trying all 2**16 possibilities and looking for
# one that produces the most character's from alphabet

PT = ('x'*randint(0, 20)) + ('A' * 14)
plaintext, bestseed = crack_mt19937_encrypt(mt19937_encrypt(PT))

print(plaintext.decode())

.
New Best:  2623, 0
.
New Best:  6064, 2
New Best:  6860, 4
.
.
.
.
.
.
.
.
.
.
.
.
New Best:  56862, 32
.
.
xxxxxxxxxxxxxxxxxxAAAAAAAAAAAAAA


In [55]:
# %% Part 2 - password reset token

# Gen a password random password reset token created with an mt19937 RNG
# seeded with timestamp

In [56]:
def mt19937_encrypt(plaintext):

    seed = randint(0, 2**16 - 1)
    myRNG = mt.mt19937(seed)

    if isinstance(plaintext, str):
        pt_bytes = plaintext.encode()
    elif isinstance(plaintext, int):
        pt_bytes = plaintext.to_bytes()
    else:
        raise ValueError('Not sure what to do with this plaintext data type')

    pt_len = len(pt_bytes)

    n_words = math.ceil(pt_len/4)

    key_stream = []
    for __ in range(n_words):
        key_stream += myRNG.extract_number().to_bytes(4, 'little')

    ciphertext = bitwise_xor(pt_bytes, key_stream[0:pt_len])
    return(ciphertext)

In [57]:
def check_for_mt19937(byte_list, max_look_back=1000):

    current_time = int(time.time())
    start_time = current_time - max_look_back
    num_words = math.ceil(len(byte_list)/4)

    for seed in range(start_time, current_time):

        myMT = mt.mt19937(seed)
        mt_out_little = []
        mt_out_big = []

        for ii in range(num_words):

            rand_out = myMT.extract_number()
            mt_out_little += rand_out.to_bytes(4, 'little')
            mt_out_big += rand_out.to_bytes(4, 'big')

        if (byte_list == mt_out_little) or (byte_list == mt_out_big):

            print(f"Found Match - Token is mt19937 with seed = {seed}")
            return(seed)

    return(-1)

In [58]:
token, seed = gen_mt19937_token()
time.sleep(1)

seed = check_for_mt19937(token, 5000)

Found Match - Token is mt19937 with seed = 1584561550


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)